In [5]:
import pandas as pd
import numpy as np

1 40 57 44 45 21 10 55 59 46 52 61


In [226]:
# 앙상블을 위한 데이터 경로
FILE_PATH = "/opt/ml/level2_klue_nlp-level2-nlp-11/dataset/prediction/ensemble/"
# 앙상블 결과 저장 이름
ENSEMBLE_CSV_NAME = "ensemble12_final5"
# 앙상블 할 모델 이름 리스트
# csv_list = ["output (1)", "output (10)", "output (21)", "output (40)", "output (44)", "output (45)", "output (46)",
#             "output (52)", "output (55)", "output (57)", "output (59)", "output (61)"]
csv_list = ["output (1)", "output (61)", "output (40)", "output (57)", "output (44)", "output (55)", "output (21)",
            "output (59)", "output (51)", "output (46)", "output (53)", "output (45)"]
# 가중치
# weight = [0.5, 0.5]

In [227]:
print(len(csv_list))

12


In [228]:
BALANCED_WEIGHT = [1/12] *12
print(BALANCED_WEIGHT)

[0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]


In [230]:
BALANCED_WEIGHT = [1/12] *12
# BALANCED_WEIGHT[] += 0.0000000000000002
print(sum(BALANCED_WEIGHT))
# assert len(csv_list) == len(weight), "csv_list와 weight의 길이가 다릅니다."
# assert sum(weight) == 1, "가중치의 합은 1이어야 합니다."
assert sum(BALANCED_WEIGHT) == 1

1.0


In [231]:
def get_csv(csv_list):
    for csv in csv_list:
        try:
            yield pd.read_csv(FILE_PATH + csv + ".csv")
        except FileNotFoundError:
            print("파일을 찾을 수 없습니다. 파일명을 확인해주세요.")
            break

df_list = list(get_csv(csv_list))
print(f"앙상블 할 csv 파일 개수 : {len(df_list)}\n앙상블 할 csv 파일 : {csv_list}")

앙상블 할 csv 파일 개수 : 12
앙상블 할 csv 파일 : ['output (1)', 'output (61)', 'output (40)', 'output (57)', 'output (44)', 'output (55)', 'output (21)', 'output (59)', 'output (51)', 'output (46)', 'output (53)', 'output (45)']


In [232]:
# def probs_to_list(probs):
#     return np.array([float(prob) for prob in probs[1:-1].split(",")])

# df_list = [df.assign(probs=df["probs"].apply(probs_to_list)) for df in df_list]


# df["probs"] = df["probs"].apply(probs_to_list)
# df_2["probs"] = df_2["probs"].apply(probs_to_list)


# df['probs'] = df.probs.apply(lambda x: x[1:-1].split(','))
# df['probs'] = df.probs.apply(lambda x: [float(i) for i in x])

In [233]:
def probs_to_list(probs):
    return np.array([float(prob) for prob in probs[1:-1].split(",")])

df_list = [df.assign(probs=df["probs"].apply(probs_to_list)) for df in df_list]
probs_list = [df.probs.values.tolist() for df in df_list]

In [234]:
def set_weight(probs_list, weight=BALANCED_WEIGHT):
    new_probs_list = []
    for i in range(len(probs_list)):
        new_probs_list.append((np.array(probs_list[i]) * weight[i]).tolist())
    return new_probs_list

probs_weighted_list = set_weight(probs_list)

In [235]:
# # 가중치가 없던 예전 버전 : deprecated
# def get_average(probs_list):
#     df_probs_sum = np.sum(probs_list, axis=0)
#     return df_probs_sum / len(probs_list)

# probs_average_deprecated = get_average(probs_list).tolist()

In [236]:
def get_average(probs_list):
    return np.sum(probs_list, axis=0).tolist()

probs_average = get_average(probs_weighted_list)

In [237]:
# assert probs_average[3] == probs_average_deprecated[3]

In [238]:
# assert probs_average[0] == (df_list[0].probs[0] + df_list[1].probs[0]) / 2
# random_num = 125
# average = (df_list[0].probs[random_num] + df_list[1].probs[random_num] + df_list[2].probs[random_num]
#            + df_list[3].probs[random_num] + df_list[4].probs[random_num]+df_list[5].probs[random_num]
#            +df_list[6].probs[random_num]+df_list[7].probs[random_num]+df_list[8].probs[random_num]+df_list[9].probs[random_num]) / 10
# assert np.allclose(probs_average[random_num], average)

In [239]:
result = [np.argmax(prob) for prob in probs_average]

assert len(df_list[0]) == len(result)

In [240]:
import pickle
def label_to_num(label: np.ndarray) -> list:
    num_label = []
    with open("/opt/ml/code/dict_label_to_num.pkl", "rb") as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label


def num_to_label(label: np.ndarray) -> list:
    """
    숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
    """
    origin_label = []
    with open("/opt/ml/code/dict_num_to_label.pkl", "rb") as f:
        dict_num_to_label = pickle.load(f)
    for v in label:
        origin_label.append(dict_num_to_label[v])

    return origin_label

In [241]:
result_label = num_to_label(result)

In [242]:
result_label[:3]

['org:product', 'per:alternate_names', 'no_relation']

In [243]:
output = pd.DataFrame({"id": df_list[0].id, "pred_label": result_label, "probs": probs_average})
output.to_csv(f"{FILE_PATH}/{ENSEMBLE_CSV_NAME}.csv", index=False)